In [1]:
import json, requests
import pandas as pd
import numpy as np
import io
import random
import time

from random import sample
import warnings
warnings.filterwarnings('ignore')

  # Libraries for stock data
import yfinance as yf

In [2]:
from transform_functions import stock_data_pull, fill_blanks, feature_create

2023-01-11 20:47:09.372381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 20:47:09.492279: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-11 20:47:13.136665: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/cuda/lib

In [3]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tckr = table[0]
# convert to list
basket = tckr.Symbol.unique().tolist()

In [4]:
bkt_12 = sample(basket,12)

**Define parameters**

In [5]:
window = 20  
features = ['return', 'vol', 'mom', 'sma', 'min', 'max']  
lags = 6

In [6]:
bkt_12

['PNC',
 'BMY',
 'BK',
 'TFX',
 'CSGP',
 'COO',
 'COF',
 'BDX',
 'BWA',
 'CLX',
 'PFE',
 'DE']

**Call functions**

In [7]:
# Pull stock data for given ticker symvols
stock_total = stock_data_pull(bkt_12,False)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Name'], dtype='object')


In [8]:
stock_total.head()

,Datetime,Close,Volume,Name
0,2022-12-12 09:30:00,152.850006,52611,PNC
1,2022-12-12 09:35:00,152.320007,17405,PNC
2,2022-12-12 09:40:00,152.710007,20228,PNC
3,2022-12-12 09:45:00,153.554993,14229,PNC
4,2022-12-12 09:50:00,153.250000,14318,PNC


In [11]:
date_range = stock_total.Datetime.drop_duplicates()
date_range = pd.DataFrame(date_range)
stock_ready = fill_blanks(stock_total,'Name',date_range,'Datetime');

**Feature Create**

In [12]:
# call function to genereate all new variables
stk_rdy_2, cols =  feature_create(stock_ready,bkt_12, window,lags,features)

all good, all data contains 1636 observations
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616
size of temp is 1616


In [14]:
#cols

In [16]:
stk_rdy_2.columns

Index(['Datetime', 'Close', 'Volume', 'Name', 'return', 'vol', 'mom', 'sma',
       'min', 'max', 'return_lag_1', 'return_lag_2', 'return_lag_3',
       'return_lag_4', 'return_lag_5', 'return_lag_6', 'vol_lag_1',
       'vol_lag_2', 'vol_lag_3', 'vol_lag_4', 'vol_lag_5', 'vol_lag_6',
       'mom_lag_1', 'mom_lag_2', 'mom_lag_3', 'mom_lag_4', 'mom_lag_5',
       'mom_lag_6', 'sma_lag_1', 'sma_lag_2', 'sma_lag_3', 'sma_lag_4',
       'sma_lag_5', 'sma_lag_6', 'min_lag_1', 'min_lag_2', 'min_lag_3',
       'min_lag_4', 'min_lag_5', 'min_lag_6', 'max_lag_1', 'max_lag_2',
       'max_lag_3', 'max_lag_4', 'max_lag_5', 'max_lag_6', 'direction'],
      dtype='object')

In [17]:
input_ds = stk_rdy_2[cols]

In [18]:
input_ds.columns

Index(['return_lag_1', 'return_lag_2', 'return_lag_3', 'return_lag_4',
       'return_lag_5', 'return_lag_6', 'vol_lag_1', 'vol_lag_2', 'vol_lag_3',
       'vol_lag_4', 'vol_lag_5', 'vol_lag_6', 'mom_lag_1', 'mom_lag_2',
       'mom_lag_3', 'mom_lag_4', 'mom_lag_5', 'mom_lag_6', 'sma_lag_1',
       'sma_lag_2', 'sma_lag_3', 'sma_lag_4', 'sma_lag_5', 'sma_lag_6',
       'min_lag_1', 'min_lag_2', 'min_lag_3', 'min_lag_4', 'min_lag_5',
       'min_lag_6', 'max_lag_1', 'max_lag_2', 'max_lag_3', 'max_lag_4',
       'max_lag_5', 'max_lag_6'],
      dtype='object')

In [19]:
input_ds.iloc[:10,:]

,return_lag_1,return_lag_2,return_lag_3,return_lag_4,return_lag_5,return_lag_6,vol_lag_1,vol_lag_2,vol_lag_3,vol_lag_4,...,min_lag_3,min_lag_4,min_lag_5,min_lag_6,max_lag_1,max_lag_2,max_lag_3,max_lag_4,max_lag_5,max_lag_6
6,0.000550,-0.002100,0.000840,0.001584,0.000939,-0.001618,0.001572,0.001666,0.001658,0.002009,...,153.100006,153.100006,152.710007,152.320007,154.919998,154.919998,154.919998,154.789993,154.649994,154.649994
7,-0.001197,0.000550,-0.002100,0.000840,0.001584,0.000939,0.001603,0.001572,0.001666,0.001658,...,153.100006,153.100006,153.100006,152.710007,154.919998,154.919998,154.919998,154.919998,154.789993,154.649994
8,0.001261,-0.001197,0.000550,-0.002100,0.000840,0.001584,0.001613,0.001603,0.001572,0.001666,...,153.100006,153.100006,153.100006,153.100006,154.919998,154.919998,154.919998,154.919998,154.919998,154.789993
9,-0.000365,0.001261,-0.001197,0.000550,-0.002100,0.000840,0.001621,0.001613,0.001603,0.001572,...,153.100006,153.100006,153.100006,153.100006,154.919998,154.919998,154.919998,154.919998,154.919998,154.919998
10,0.001269,-0.000365,0.001261,-0.001197,0.000550,-0.002100,0.001360,0.001621,0.001613,0.001603,...,153.100006,153.100006,153.100006,153.100006,154.919998,154.919998,154.919998,154.919998,154.919998,154.919998
11,0.001033,0.001269,-0.000365,0.001261,-0.001197,0.000550,0.001203,0.001360,0.001621,0.001613,...,153.100006,153.100006,153.100006,153.100006,154.990005,154.919998,154.919998,154.919998,154.919998,154.919998
12,-0.000194,0.001033,0.001269,-0.000365,0.001261,-0.001197,0.001207,0.001203,0.001360,0.001621,...,153.479996,153.100006,153.100006,153.100006,154.990005,154.990005,154.919998,154.919998,154.919998,154.919998
13,0.000806,-0.000194,0.001033,0.001269,-0.000365,0.001261,0.001210,0.001207,0.001203,0.001360,...,153.479996,153.479996,153.100006,153.100006,155.085007,154.990005,154.990005,154.919998,154.919998,154.919998
14,0.002544,0.000806,-0.000194,0.001033,0.001269,-0.000365,0.001229,0.001210,0.001207,0.001203,...,153.479996,153.479996,153.479996,153.100006,155.479996,155.085007,154.990005,154.990005,154.919998,154.919998
15,-0.001996,0.002544,0.000806,-0.000194,0.001033,0.001269,0.001324,0.001229,0.001210,0.001207,...,153.479996,153.479996,153.479996,153.479996,155.479996,155.479996,155.085007,154.990005,154.990005,154.919998
